In [ ]:
#importing packages
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#mounting google drive on colab notebook
from google.colab import drive
drive.mount('/d')


Drive already mounted at /d; to attempt to forcibly remount, call drive.mount("/d", force_remount=True).


In [ ]:
#reading data from csv file
data = pd.read_csv("/d/My Drive/Master's project code_Arun/2007/2007/Formatted Files/xml_test_formatted.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#checking the number of classes in the data
class_set = set(data['class'].tolist())
print(class_set)

{' OsCommanding', ' Valid', ' XPathInjection', ' SSI', ' LdapInjection', ' XSS', ' PathTransversal', ' SqlInjection'}


In [ ]:
#extracting only 600 rows of data belonging to each class
data1 = data.loc[data['class'].isin(class_set)].groupby('class').head(500)
print(data1)


       Unnamed: 0  ... PUT Body
0               0  ...      NaN
1               1  ...      NaN
2               2  ...      NaN
3               3  ...      NaN
4               4  ...      NaN
...           ...  ...      ...
11312       11312  ...      NaN
11313       11313  ...      NaN
11315       11315  ...      NaN
11319       11319  ...      NaN
11320       11320  ...      NaN

[4000 rows x 59 columns]


In [ ]:
#extact only 6 columns to a new dataframe
X = data1.loc[:,['GET','POST','POST Body','PUT','PUT Body']]
y = data1.loc[:,['class']]

In [ ]:
#fil the nans with 0
X.fillna(0, inplace = True)
X = X.astype(str)

In [ ]:
X.head(10)

,GET,POST,POST Body,PUT,PUT Body
0,/lqlehaRus4/wREtSesTncl9tln/nI/u8ti8/L@AHchild...,0,0,0,0
1,/dcNF_symcnzynma/en46/tQrU2n/o10jR86coK.sDGjMe...,0,0,0,0
2,/ea@l/tOnimc/yf4s-cmdh/e7Ag2mql9.jpeg?uOh=sDSQ...,0,0,0,0
3,0,/8WvHZW3f4TMgxH_/sitdtOffnshyUettq/WO2lInQSFF0...,Max-Forwardsnannototpsfd=deleteCn&a2trserdDSne...,0,0
4,0,/hotdekltascs/d-N/tettr8etRktst/z9minput4EqR35...,nannlmbmpea=eSddetgeciimsibn&aiftr=39&mst=1178...,0,0
5,/aKdNFrwg3/nBdttln/rJbsyUwtV7gL05HUIU/IrE/libe...,0,0,0,0
6,/qsosehtoaauXrmw2e/Usg.htm?1TphdpiestvReh=%29+...,0,0,0,0
7,0,0,0,/orbeRmTnNyoet2urere/toR/aUJ/G_YhavingmGAunall...,nanF@b8inputoptRallbF=rse&gaeonrnshyss2=zs&lLP...
8,/eiy.asp?XuRPe=Rtmsr&ph5dd=u0h1otperlSe&sex2tp...,0,0,0,0
9,/AzYnph-J2echo4Rn/esIi83.tsvoj4J/oqtiEzhrns7vy...,0,0,0,0


In [ ]:
#combining the GET, POST and PUT columns into one column
#combining the POST/PUT Body columns into one column
#this is done for ease of computation
X_combo = pd.DataFrame(columns = ['GET/POST/PUT','POST/PUT Body'])
for i,rows in X.iterrows():
  if rows[0] !='0':
    get_value = rows[0]
    body_value = '0'
  elif rows[1] != '0':
    get_value = rows[1]
    body_value = rows[2]
  else:
    get_value = rows[3]
    body_value = rows[4]
  X_combo = X_combo.append({'GET/POST/PUT' : get_value,'POST/PUT Body' :body_value}, ignore_index = True)

In [ ]:
#the data in both the columns are decoded
from urllib.parse import unquote
for i,rows in X_combo.iterrows():
    if rows[0] != 0 :
        rows[0] = unquote(rows[0])
    if rows[1] != 0:
        rows[1] = unquote(rows[1])
  

In [ ]:
#Bag of Words implementation
vectorizer = CountVectorizer()  
vectorizer1 = CountVectorizer()


In [ ]:
#BOW for GET/POST/PUT column
vectorizer.fit(X_combo['GET/POST/PUT'])
vector = vectorizer.transform(X_combo['GET/POST/PUT']).toarray()

In [ ]:
#BOW for POST Body/PUT Body column
vectorizer1.fit(X_combo['POST/PUT Body'])
vector1 = vectorizer1.transform(X_combo['POST/PUT Body']).toarray()

In [ ]:
#combining the BOW outputs of both the columns
X_train = np.concatenate((vector,vector1),axis = 1)

#encoding the class labels into integers
label_encoder = LabelEncoder()
y_train = np.array(label_encoder.fit_transform(y['class']))

In [ ]:
#checking shape of the arrays before training
print(X_train.shape)
print(y_train.shape)


(4000, 77625)
(4000,)


In [ ]:
#splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
#Model 1: Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("Random Forest Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Random Forest Classifier :
Accuracy : 0.75
Precision Score : 0.834826537307475
Recall score : 0.75
F1 - score : 0.772658881697371
Confusion Matrix :
 [[63  1  9  1  2 40  0  0]
 [ 1 83  7  0  0 12  1  0]
 [ 0  1 91  0  1 22  0  0]
 [ 1  0  6 80  0 12  0  0]
 [ 0  3  2  1 75 16  0  0]
 [ 0  2 28  0  1 57  1  1]
 [ 0  1  3  0  0 13 77  0]
 [ 0  2  5  0  0  4  0 74]]
Classification Report :
                   precision    recall  f1-score   support

    OsCommanding       0.97      0.54      0.70       116
           Valid       0.89      0.80      0.84       104
  XPathInjection       0.60      0.79      0.68       115
             SSI       0.98      0.81      0.88        99
   LdapInjection       0.95      0.77      0.85        97
             XSS       0.32      0.63      0.43        90
 PathTransversal       0.97      0.82      0.89        94
    SqlInjection       0.99      0.87      0.93        85

        accuracy                           0.75       800
       macro avg       0.8

In [ ]:
#model 2 : Logistic Regression
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.7575


In [ ]:
print("Logistic Regression Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Logistic Regression Classifier :
Accuracy : 0.7575
Precision Score : 0.8627207397933782
Recall score : 0.7575
F1 - score : 0.7831560544074654
Confusion Matrix :
 [[60  1  2  0  1 52  0  0]
 [ 0 78  9  1  0 16  0  0]
 [ 0  0 89  0  1 24  1  0]
 [ 0  0  2 80  0 16  0  1]
 [ 0  4  1  1 74 17  0  0]
 [ 0  3 11  0  0 74  1  1]
 [ 0  1  1  0  0 15 77  0]
 [ 0  2  2  0  0  7  0 74]]
Classification Report :
                   precision    recall  f1-score   support

    OsCommanding       1.00      0.52      0.68       116
           Valid       0.88      0.75      0.81       104
  XPathInjection       0.76      0.77      0.77       115
             SSI       0.98      0.81      0.88        99
   LdapInjection       0.97      0.76      0.86        97
             XSS       0.33      0.82      0.48        90
 PathTransversal       0.97      0.82      0.89        94
    SqlInjection       0.97      0.87      0.92        85

        accuracy                           0.76       800
       macro a

In [ ]:
#Model 3: Stochastic Gradient Descent Classifier
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.76375


In [ ]:
print("Stochastic Gradient Descent Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Stochastic Gradient Descent Classifier :
Accuracy : 0.76375
Precision Score : 0.8190128727852138
Recall score : 0.76375
F1 - score : 0.7759684683697489
Confusion Matrix :
 [[63  0  1  1  2 37 12  0]
 [ 0 80  9  1  1 10  3  0]
 [ 1  3 89  0  1 16  5  0]
 [ 0  0  1 81  3 12  2  0]
 [ 1  4  0  3 76 12  0  1]
 [ 2  2  8  0  0 69  7  2]
 [ 1  0  0  1  1 12 79  0]
 [ 1  2  2  0  1  4  1 74]]
Classification Report :
                   precision    recall  f1-score   support

    OsCommanding       0.91      0.54      0.68       116
           Valid       0.88      0.77      0.82       104
  XPathInjection       0.81      0.77      0.79       115
             SSI       0.93      0.82      0.87        99
   LdapInjection       0.89      0.78      0.84        97
             XSS       0.40      0.77      0.53        90
 PathTransversal       0.72      0.84      0.78        94
    SqlInjection       0.96      0.87      0.91        85

        accuracy                           0.76       800
    

In [ ]:
#Model4: Decision Tree Classifier
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.77375


In [ ]:
print("Decision Tree Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Decision Tree Classifier :
Accuracy : 0.77375
Precision Score : 0.8545835579145802
Recall score : 0.77375
F1 - score : 0.7954007252730676
Confusion Matrix :
 [[64  0  1  0  3 46  1  1]
 [ 2 86  2  0  0 14  0  0]
 [ 2  2 91  0  1 18  1  0]
 [ 0  0  0 80  2 16  0  1]
 [ 2  1  1  1 76 15  0  1]
 [ 0  4 11  0  1 71  1  2]
 [ 0  0  1  0  0 16 77  0]
 [ 0  2  3  0  0  6  0 74]]
Classification Report :
                   precision    recall  f1-score   support

    OsCommanding       0.91      0.55      0.69       116
           Valid       0.91      0.83      0.86       104
  XPathInjection       0.83      0.79      0.81       115
             SSI       0.99      0.81      0.89        99
   LdapInjection       0.92      0.78      0.84        97
             XSS       0.35      0.79      0.49        90
 PathTransversal       0.96      0.82      0.89        94
    SqlInjection       0.94      0.87      0.90        85

        accuracy                           0.77       800
       macro avg  

In [ ]:
 from sklearn.neural_network import MLPClassifier
 model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
 model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("MLP Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

MLP Classifier :
Accuracy : 0.53875
Precision Score : 0.5073637364902269
Recall score : 0.53875
F1 - score : 0.4781082742098734
Confusion Matrix :
 [[  0   0  41   3   4   0  67   1]
 [  0  62  24  14   3   0   1   0]
 [  0   1 102   3   2   2   5   0]
 [  0   6  13  72   6   0   2   0]
 [  0   3  23   5  47   0  17   2]
 [  0   0  66   7   8   1   7   1]
 [  0   0  11   4   1   0  78   0]
 [  0   1   9   0   6   0   0  69]]
Classification Report :
                   precision    recall  f1-score   support

    OsCommanding       0.00      0.00      0.00       116
           Valid       0.85      0.60      0.70       104
  XPathInjection       0.35      0.89      0.50       115
             SSI       0.67      0.73      0.70        99
   LdapInjection       0.61      0.48      0.54        97
             XSS       0.33      0.01      0.02        90
 PathTransversal       0.44      0.83      0.58        94
    SqlInjection       0.95      0.81      0.87        85

        accuracy      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
